# 입력과 출력

In [4]:
:t putStrLn
:t putStrLn "Hello, world"
-- ()는 빈 튜플이며 Unit이라고 읽는다 () 타입이며, 값도 ()이다

putStrLn :: String -> IO ()

putStrLn "Hello, world" :: IO ()

IO 작업은 부수적인 작업과 함께 실행되며, 또한 어떤 결과를 나타낸다.

IO 작업이 그 결과에 굴복(yield)된다고 말한다.
터미널에 문자열을 출력하는 것은 ()의 더미 값이 사용되므로 의미있는 반환값을 갖지 않는다.

## do notation

In [8]:
main = do
    putStrLn "Hello, what's your name?"
    name <- getLine
    -- IO작업을 수행한 후, 그 결과값을 name에 바인딩
    putStrLn $ "Hey " ++ name ++ ", you rock!"
    -- 마지막 작업은 바인딩 할 수 없다. 13장 Monad에서 확인하자.
    -- 마지막 작업에서 값을 자동으로 추출하여 그것의 결과를 만든다.
    
main

Hello, what's your name?
Hey Kim Juho, you rock!

In [9]:
:t getLine

getLine :: IO String

IO 작업의 결과물을 사용하기 위해서는 `<-`를 사용한다.

IO 작업에 대한 데이터를 얻으려고 한다면, IO작업 내에 있을 때만 가능하다. 이는 코드의 완전한 부분과 불안전한 부분을 깔끔하게 나누기 위해서 하스켈이 관리하는 방법이다.

즉슨, 순수하지 않은 작업은 순수하지 않은 환경에서 하라는 이야기

모든 IO작업은 결과를 가지므로, `foo <- putStrLn "hello~ whats..."` 와 같이 작성할 수 있으나, foo == ()일것이므로 무의미하다.

In [12]:
myLine = getLine
:t myLine
-- IO가 언박싱되지 않는다.

myLine :: IO String

IO 작업은 main의 이름이 주어지거나 do 블록으로 구성된 더 큰 IO작업 내에 있을 떄 수행될 것

```
01-helloworld.hs:1:1: error:
    The IO action ‘main’ is not defined in module ‘Main’
  |
1 | main1 = putStrLn "hello, world"
  | ^
```

## let in IO

In [14]:
-- 값과 이름을 바인딩 시키기 위해 let 구문을 사용할 수 있다.

import Data.Char

main = do
    putStrLn "What's your first name"
    firstName <- getLine
    putStrLn "What's your last name"
    lastName <- getLine
    let bigFirstName = map toUpper firstName
        bigLastName = map toUpper lastName
    putStrLn $ "hey " ++ bigFirstName ++ " " ++ bigLastName ++ ", how are you?"
    
main

-- <-는 IO작업을 수행하고 그결과를 이름에 바인딩하는것
-- = 는 

What's your first name
What's your last name
hey JUHO KIM, how are you?

In [16]:
-- 거꾸로 넣기

main = do
    line <- getLine
    if null line
    -- if 표현식이 마지막 IO값이므로 then 이던, else이던 IO 값을 나타내야한다.
        then return ()
        -- return은 인자 값을 IO로 매핑한다
        -- 가짜 IO 작업을 만든셈
        else do
        -- IO 값을 만들기 위해 do notation을 활용한다.
            putStrLn $ reverseWords line
            -- 출력
            main 
            -- 재귀
            -- 현재 main은 do notation을 활용하는, 그 자체로 IO 작업
            
reverseWords :: String -> String
reverseWords = unwords . map reverse . words
--unwords : space join와 함께 [String] -> String 함

main

naelc pu no elsia rebmun enin

In [17]:
:t return

return :: forall (m :: * -> *) a. Monad m => a -> m a

In [18]:
-- 하스켈의 return이 함수를 끝내지는 않는다

main = do  
    return ()  
    return "HAHAHA"  
    line <- getLine  
    return "BLAH BLAH BLAH"  
    return 4  
    putStrLn line  
    
main

Line 4: Redundant return
Found:
return ()
Why not:
Line 5: Redundant return
Found:
return "HAHAHA"
Why not:
Line 7: Redundant return
Found:
return "BLAH BLAH BLAH"
Why not:
Line 8: Redundant return
Found:
return 4
Why not:

$Xt

In [20]:
-- return은 IO 매핑을 위하여 사용될 수 있다.

main = do  
    a <- return "hell"  
    b <- return "yeah!"  
    putStrLn $ a ++ " " ++ b  
    
main

Line 4: Use let
Found:
a <- return "hell"
Why not:
let a = "hell"Line 5: Use let
Found:
b <- return "yeah!"
Why not:
let b = "yeah!"

hell yeah!

In [22]:
main = do  
    let a = "hell"  
        b = "yeah"  
    putStrLn $ a ++ " " ++ b  
main

hell yeah

## IO 함수들

In [24]:
main = do   putStr "Hey, "  
            putStr "I'm "  
            putStrLn "Andy!"   
main

Hey, I'm Andy!

In [25]:
main = do   putChar 't'  
            putChar 'e'  
            putChar 'h'  
main

teh

In [27]:
main = do   print True  
            print 2  
            print "haha"  
            print 3.2  
            print [3,4,3]  
main

True
2
"haha"
3.2
[3,4,3]

In [31]:
import qualified Control.Monad as Monad
  
main = do  
    c <- getChar  
    Monad.when (c /= ' ') $ do  
    -- 흐름제어 구문처럼 보이지만 일반적인 함수
    -- Bool 과 IO 작업을 받아 Bool이 True 이면 제공받은 IO 작업을 반환하며, False일 때는 return ()를 반환
        putChar c  
        main 
        --재귀
main


y
e
s

In [34]:
main = do  
    rs <- sequence [getLine, getLine, getLine]  
    -- IO 작업 리스트를 하나씩 **실행**한 결과 리스트로 만드는 것
    -- 단순히 map print [1,2,3,4] 는 IO 리스트를 만들뿐, 결과가 나오진 않음
    print rs  
main

["1","2","3"]

In [35]:
sequence $ map print [1,2,3,4]

Line 1: Use mapM
Found:
sequence $ map print [1, 2, 3, 4]
Why not:
mapM print [1, 2, 3, 4]

1
2
3
4
[(),(),(),()]

In [38]:
mapM print [1,2,3]  
-- 1) IO 작업을 반환하는 함수를 리스트에 매핑한 후
-- 2) 시퀀싱(sequence) 하는 것
-- 이 일반적이므로 mapM이 그것을 한다. (유틸리티 함수)
mapM_ print [1,2,3]  -- 뒤에 값을 버림

1
2
3
[(),(),()]

1
2
3

In [ ]:
import qualified Control.Monad as Monad
import qualified Data.Char as Char
  
main = Monad.forever $ do  
    -- IO 작업을 받아서 영원히 해당 작업을 반복함
    putStr "Give me some input: "  
    l <- getLine  
    putStrLn $ map Char.toUpper l  
    
main

Give me some input: 1
Give me some input: 1
Give me some input: 1
Give me some input: 1
Give me some input: 1
Give me some input: 2
Give me some input: Press Ctrl-C again to quit kernel.

In [1]:
import qualified Control.Monad as Monad
  
main = do   
    colors <- Monad.forM [1,2,3,4] (\a -> do  
    -- mapM과 매개변수 순서가 반대인 함수
    -- 함수가 2번째로 오기 때문에, 람다 표기법을 더 이쁘게 쓸 수 있음
        putStrLn $ "Which color do you associate with the number " ++ show a ++ "?"  
        color <- getLine  
        return color)  
    putStrLn "The colors that you associate with 1, 2, 3 and 4 are: "  
    Monad.mapM putStrLn colors  
    
main

Line 4: Redundant return
Found:
do putStrLn
     $ "Which color do you associate with the number " ++ show a ++ "?"
   color <- getLine
   return color
Why not:
do putStrLn
     $ "Which color do you associate with the number " ++ show a ++ "?"
   getLine

Which color do you associate with the number 1?
Which color do you associate with the number 2?
Which color do you associate with the number 3?
Which color do you associate with the number 4?
The colors that you associate with 1, 2, 3 and 4 are: 
red
blue
white
black
[(),(),(),()]

## Review

IO 작업은 값이며, 하스켈의 다른 값과 유사하다.

IO 작업을 매개변수처럼 함수에 전달할 수 있으며, 함수는 처리 결과로 I/O 작업을 반환할 수 있다.

예외로 main 함수이거나 GHCi에서는 동작한다.